In [1]:
import os
import pandas as pd
from tqdm import tqdm
import nltk
nltk.download('vader_lexicon')
from multiprocessing import Pool, cpu_count


from nltk.sentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/aunno/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
stocks = ['AAPL', 'GME', 'MCD', 'MSFT', 'NFLX', 'NVDA', 'TSLA']

In [3]:
#Cleaning and merging POSTS files
posts_file_path=os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"post_data","posts.csv")
all_posts_df= pd.read_csv(posts_file_path)

stock_index_file_path=os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"post_data","stock_index.csv")
stock_index_df= pd.read_csv(stock_index_file_path)

all_posts_df


,id,created_utc,subreddit,author,title,selftext,permalink,url
0,7nca41,1514768665,wallstreetbets,alreadyinuse5000,Blowing versus sucking,AAPL just entered a contract to purchase 51 of...,/r/wallstreetbets/comments/7nca41/blowing_vers...,https://www.reddit.com/r/wallstreetbets/commen...
1,7ncxcn,1514776674,pcmasterrace,castatech,Why is it that the new Windows 10 picture view...,What kind of an upgrade was this? It just suck...,/r/pcmasterrace/comments/7ncxcn/why_is_it_that...,https://www.reddit.com/r/pcmasterrace/comments...
2,7ndtyy,1514789593,TIMESINDIAauto,AutoNewsAdmin,[Business] - New year looks grim for McD’s staff,NaN,/r/TIMESINDIAauto/comments/7ndtyy/business_new...,https://timesofindia.indiatimes.com/business/i...
3,7ndu9l,1514789727,AutoNewspaper,AutoNewspaperAdmin,[Business] - New year looks grim for McD’s sta...,NaN,/r/AutoNewspaper/comments/7ndu9l/business_new_...,https://timesofindia.indiatimes.com/business/i...
4,7neolz,1514805270,teslamotors,AutoModerator,Bi-weekly TSLA Investor Thread,This will post every other Monday (EST) at 6AM...,/r/teslamotors/comments/7neolz/biweekly_tsla_i...,https://www.reddit.com/r/teslamotors/comments/...
...,...,...,...,...,...,...,...,...
727177,1006ago,1672528591,InsiderMemeTrading,InsiderMemeBot,"LEADERBOARD: Sat, Dec 31, 2022: 06:16 PM EST",#TOP TRADERS \n ##Overall\nRanking | Name | ...,/r/InsiderMemeTrading/comments/1006ago/leaderb...,https://www.reddit.com/r/InsiderMemeTrading/co...
727178,1006u0d,1672530308,Superstonk,AAAJade,the DRS GME tips im giving at the Hockey game!...,NaN,/r/Superstonk/comments/1006u0d/the_drs_gme_tip...,https://i.redd.it/avmow0la3d9a1.jpg
727179,1006xh3,1672530618,u_WSBTickercountBOT,WSBTickercountBOT,Today's most mentioned tickers,## The following are the top 10 most mentioned...,/r/u_WSBTickercountBOT/comments/1006xh3/todays...,https://www.reddit.com/r/u_WSBTickercountBOT/c...
727180,1006ya3,1672530691,gme_meltdown,RunnyTinkles,GME Influencer Lacks Self-Awareness. Comments ...,NaN,/r/gme_meltdown/comments/1006ya3/gme_influence...,https://imgur.com/a/RXXWM6Y


In [4]:
stock_index_df

,id,stock_symbol,created_utc
0,7nca41,aapl,1514768665
1,7ncxcn,msft,1514776674
2,7ndtyy,mcd,1514789593
3,7ndu9l,mcd,1514789727
4,7neolz,tsla,1514805270
...,...,...,...
760092,1006xh3,nvda,1672530618
760093,1006xh3,tsla,1672530618
760094,1006xh3,aapl,1672530618
760095,1006ya3,gme,1672530691


In [5]:
#making date proper
all_posts_df['created_utc'] = pd.to_datetime(all_posts_df['created_utc'], unit='s')
stock_index_df['created_utc'] = pd.to_datetime(stock_index_df['created_utc'], unit='s')
#merging on basis of id,time
post_stock_df= pd.merge(all_posts_df, stock_index_df, on=['id'], how='inner')


post_stock_df=post_stock_df.drop(columns= ['subreddit', 'author', 
        'permalink', 'url','created_utc_y'])
post_stock_df=post_stock_df.rename(columns={'created_utc_x': 'created_at'})
post_stock_df=post_stock_df.sort_values(by=['stock_symbol','created_at'])

post_stock_df

,id,created_at,title,selftext,stock_symbol
0,7nca41,2018-01-01 01:04:25,Blowing versus sucking,AAPL just entered a contract to purchase 51 of...,aapl
7,7nfzp3,2018-01-01 16:37:41,The 2018 /r/Robinhood Stock Picking Game,# tl;dr\n\n - Stock picking game will last all...,aapl
13,7nhc2r,2018-01-01 20:21:33,Hesitant to invest in $AAPL,Looking at AAPLs fundamentals and the pile of ...,aapl
18,7nhwud,2018-01-01 21:54:14,Stock mix help required (ETF),I’ve decided I’m most likely interested in jus...,aapl
21,7ni0o7,2018-01-01 22:12:01,ETF advice please!,I’ve decided I’m most likely interested in jus...,aapl
...,...,...,...,...,...
760078,1003u7g,2022-12-31 21:13:49,$TSLA Awaiting Short Signal based off 9 signal...,NaN,tsla
760084,1004hze,2022-12-31 21:45:57,2022 Performance and Lessons,Doing this in post form so I can include scree...,tsla
760088,1005u7z,2022-12-31 22:53:13,2022 Berry Bad Bear year in review,"Tough year for many, myself included. My tradi...",tsla
760090,1006ago,2022-12-31 23:16:31,"LEADERBOARD: Sat, Dec 31, 2022: 06:16 PM EST",#TOP TRADERS \n ##Overall\nRanking | Name | ...,tsla


In [7]:
apple_price_file_path=os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"price_data","AAPL.csv")
apple_price_df = pd.read_csv(apple_price_file_path)
apple_price_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12 00:00:00-05:00,0.098834,0.099264,0.098834,0.098834,469033600,0.00,0.0
1,1980-12-15 00:00:00-05:00,0.094108,0.094108,0.093678,0.093678,175884800,0.00,0.0
2,1980-12-16 00:00:00-05:00,0.087232,0.087232,0.086802,0.086802,105728000,0.00,0.0
3,1980-12-17 00:00:00-05:00,0.088951,0.089381,0.088951,0.088951,86441600,0.00,0.0
4,1980-12-18 00:00:00-05:00,0.091530,0.091959,0.091530,0.091530,73449600,0.00,0.0
...,...,...,...,...,...,...,...,...
11065,2024-11-04 00:00:00-05:00,220.747138,222.545148,219.468546,221.766006,44944500,0.00,0.0
11066,2024-11-05 00:00:00-05:00,221.556241,223.703872,220.896963,223.204422,28111300,0.00,0.0
11067,2024-11-06 00:00:00-05:00,222.365355,225.821559,220.946918,222.475235,54561100,0.00,0.0
11068,2024-11-07 00:00:00-05:00,224.383137,227.629565,224.323205,227.229996,42137700,0.00,0.0


In [8]:
#Correct date format
apple_price_df['Date'] = pd.to_datetime(apple_price_df['Date'], errors='coerce', utc=True)

#drop dividents and stock splits
apple_price_df.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

apple_price_df['Date'] = apple_price_df['Date'].dt.date
apple_price_df


,Date,Open,High,Low,Close,Volume
0,1980-12-12,0.098834,0.099264,0.098834,0.098834,469033600
1,1980-12-15,0.094108,0.094108,0.093678,0.093678,175884800
2,1980-12-16,0.087232,0.087232,0.086802,0.086802,105728000
3,1980-12-17,0.088951,0.089381,0.088951,0.088951,86441600
4,1980-12-18,0.091530,0.091959,0.091530,0.091530,73449600
...,...,...,...,...,...,...
11065,2024-11-04,220.747138,222.545148,219.468546,221.766006,44944500
11066,2024-11-05,221.556241,223.703872,220.896963,223.204422,28111300
11067,2024-11-06,222.365355,225.821559,220.946918,222.475235,54561100
11068,2024-11-07,224.383137,227.629565,224.323205,227.229996,42137700


In [9]:
# Filter posts related to Apple (AAPL) and format the 'created_at' column
apple_posts_df = post_stock_df[post_stock_df['stock_symbol'].str.lower() == 'aapl'].copy()
apple_posts_df['Date'] = pd.to_datetime(apple_posts_df['created_at']).dt.date
apple_posts_df.drop(columns=['created_at'], inplace=True)

# Replace NaN values in the 'selftext' column with an empty string
apple_posts_df['selftext'] = apple_posts_df['selftext'].fillna('')

# Filter Apple price data for the date range 2018-01-01 to 2022-12-31
start_date = pd.to_datetime('2018-01-01').date()
end_date = pd.to_datetime('2022-12-31').date()
apple_price_df_filtered = apple_price_df[(apple_price_df['Date'] >= start_date) & 
                                         (apple_price_df['Date'] <= end_date)]

apple_price_df_filtered

,Date,Open,High,Low,Close,Volume
9344,2018-01-02,40.030320,40.533757,39.818592,40.524345,102223600
9345,2018-01-03,40.587864,41.063072,40.453773,40.517288,118071600
9346,2018-01-04,40.590212,40.808997,40.481999,40.705486,89738400
9347,2018-01-05,40.801943,41.255976,40.710195,41.168934,94640000
9348,2018-01-08,41.016029,41.312444,40.917221,41.016029,82271200
...,...,...,...,...,...,...
10598,2022-12-23,129.557573,131.041963,128.290894,130.487793,63814900
10599,2022-12-27,130.012822,130.042509,127.380499,128.676865,69007800
10600,2022-12-28,128.320592,129.666440,124.560142,124.728371,85438400
10601,2022-12-29,126.658086,129.122172,126.400797,128.261230,75703700


In [10]:
apple_posts_df

,id,title,selftext,stock_symbol,Date
0,7nca41,Blowing versus sucking,AAPL just entered a contract to purchase 51 of...,aapl,2018-01-01
7,7nfzp3,The 2018 /r/Robinhood Stock Picking Game,# tl;dr\n\n - Stock picking game will last all...,aapl,2018-01-01
13,7nhc2r,Hesitant to invest in $AAPL,Looking at AAPLs fundamentals and the pile of ...,aapl,2018-01-01
18,7nhwud,Stock mix help required (ETF),I’ve decided I’m most likely interested in jus...,aapl,2018-01-01
21,7ni0o7,ETF advice please!,I’ve decided I’m most likely interested in jus...,aapl,2018-01-01
...,...,...,...,...,...
760075,1003mks,$AAPL Awaiting Buy Signal based off 9 signals ...,,aapl,2022-12-31
760085,1004hze,2022 Performance and Lessons,Doing this in post form so I can include scree...,aapl,2022-12-31
760089,1005u7z,2022 Berry Bad Bear year in review,"Tough year for many, myself included. My tradi...",aapl,2022-12-31
760094,1006xh3,Today's most mentioned tickers,## The following are the top 10 most mentioned...,aapl,2022-12-31


In [11]:
#merging title and selftext
apple_posts_df['text'] = apple_posts_df['title'] + '. ' + apple_posts_df['selftext']
apple_posts_df=apple_posts_df.drop(columns=['title','selftext'])
apple_posts_df

,id,stock_symbol,Date,text
0,7nca41,aapl,2018-01-01,Blowing versus sucking. AAPL just entered a co...
7,7nfzp3,aapl,2018-01-01,The 2018 /r/Robinhood Stock Picking Game. # tl...
13,7nhc2r,aapl,2018-01-01,Hesitant to invest in $AAPL. Looking at AAPLs ...
18,7nhwud,aapl,2018-01-01,Stock mix help required (ETF). I’ve decided I’...
21,7ni0o7,aapl,2018-01-01,ETF advice please!. I’ve decided I’m most like...
...,...,...,...,...
760075,1003mks,aapl,2022-12-31,$AAPL Awaiting Buy Signal based off 9 signals ...
760085,1004hze,aapl,2022-12-31,2022 Performance and Lessons. Doing this in po...
760089,1005u7z,aapl,2022-12-31,2022 Berry Bad Bear year in review. Tough year...
760094,1006xh3,aapl,2022-12-31,Today's most mentioned tickers. ## The followi...


In [12]:
def analyze_text(text):
    analyzer = SentimentIntensityAnalyzer()  # Each process gets its own instance
    scores = analyzer.polarity_scores(text)
    return scores['pos'], scores['neg'], scores['neu']

In [13]:
# Wrapper function for tqdm to work with multiprocessing
def tqdm_wrapper(func):
    def inner(data):
        return list(tqdm(func(data), total=len(data), desc="Processing Sentiments"))
    return inner

In [14]:
# Parallel sentiment analysis with progress bar
def parallel_sentiment_analysis(df, text_column):
    texts = df[text_column].tolist()  # Extract the text column as a list
    num_cores = cpu_count()
    print(f"Using {num_cores} CPU cores")

    # Initialize tqdm
    with Pool(num_cores) as pool:
        results = list(tqdm(pool.imap(analyze_text, texts), total=len(texts), desc="Sentiment Analysis"))

    # Convert the results into a DataFrame
    sentiment_df = pd.DataFrame(results, columns=['pos', 'neg', 'neu'])
    return pd.concat([df.reset_index(drop=True), sentiment_df], axis=1)

In [15]:
apple_posts_df = parallel_sentiment_analysis(apple_posts_df, 'text')
apple_posts_df

Using 12 CPU cores


Sentiment Analysis: 100%|██████████| 39306/39306 [00:37<00:00, 1034.74it/s]


,id,stock_symbol,Date,text,pos,neg,neu
0,7nca41,aapl,2018-01-01,Blowing versus sucking. AAPL just entered a co...,0.137,0.000,0.863
1,7nfzp3,aapl,2018-01-01,The 2018 /r/Robinhood Stock Picking Game. # tl...,0.072,0.028,0.900
2,7nhc2r,aapl,2018-01-01,Hesitant to invest in $AAPL. Looking at AAPLs ...,0.113,0.052,0.835
3,7nhwud,aapl,2018-01-01,Stock mix help required (ETF). I’ve decided I’...,0.137,0.000,0.863
4,7ni0o7,aapl,2018-01-01,ETF advice please!. I’ve decided I’m most like...,0.119,0.000,0.881
...,...,...,...,...,...,...,...
39301,1003mks,aapl,2022-12-31,$AAPL Awaiting Buy Signal based off 9 signals ...,0.180,0.000,0.820
39302,1004hze,aapl,2022-12-31,2022 Performance and Lessons. Doing this in po...,0.118,0.105,0.776
39303,1005u7z,aapl,2022-12-31,2022 Berry Bad Bear year in review. Tough year...,0.115,0.164,0.721
39304,1006xh3,aapl,2022-12-31,Today's most mentioned tickers. ## The followi...,0.031,0.000,0.969


In [16]:
# Remove the 'text' column
apple_posts_df = apple_posts_df.drop(columns=["text"])

# Group by 'Date' and calculate the mean of 'pos', 'neg', and 'neu'
aggregated_posts_df = apple_posts_df.groupby("Date", as_index=False)[["pos", "neg", "neu"]].mean()
aggregated_posts_df

,Date,pos,neg,neu
0,2018-01-01,0.081556,0.023889,0.894556
1,2018-01-02,0.129733,0.023333,0.846867
2,2018-01-03,0.071818,0.007000,0.921273
3,2018-01-04,0.144250,0.049125,0.806625
4,2018-01-05,0.059385,0.043308,0.897154
...,...,...,...,...
1805,2022-12-27,0.115478,0.033000,0.851522
1806,2022-12-28,0.052068,0.036750,0.911364
1807,2022-12-29,0.081450,0.045175,0.873450
1808,2022-12-30,0.084789,0.050158,0.865053


In [17]:
average_sentiments = apple_posts_df.groupby('Date')[['pos', 'neg', 'neu']].mean().reset_index()
average_sentiments.columns = ['Date', 'average_pos', 'average_neg', 'average_neu']
average_sentiments

,Date,average_pos,average_neg,average_neu
0,2018-01-01,0.081556,0.023889,0.894556
1,2018-01-02,0.129733,0.023333,0.846867
2,2018-01-03,0.071818,0.007000,0.921273
3,2018-01-04,0.144250,0.049125,0.806625
4,2018-01-05,0.059385,0.043308,0.897154
...,...,...,...,...
1805,2022-12-27,0.115478,0.033000,0.851522
1806,2022-12-28,0.052068,0.036750,0.911364
1807,2022-12-29,0.081450,0.045175,0.873450
1808,2022-12-30,0.084789,0.050158,0.865053


In [18]:
def categorize_sentiment(row):
    if row['pos'] > row['neg'] and row['pos'] > row['neu']:
        return 'positive'
    elif row['neg'] > row['pos'] and row['neg'] > row['neu']:
        return 'negative'
    else:
        return 'neutral'

In [19]:
apple_posts_df['sentiment_category'] = apple_posts_df.apply(categorize_sentiment, axis=1)
apple_posts_df

,id,stock_symbol,Date,pos,neg,neu,sentiment_category
0,7nca41,aapl,2018-01-01,0.137,0.000,0.863,neutral
1,7nfzp3,aapl,2018-01-01,0.072,0.028,0.900,neutral
2,7nhc2r,aapl,2018-01-01,0.113,0.052,0.835,neutral
3,7nhwud,aapl,2018-01-01,0.137,0.000,0.863,neutral
4,7ni0o7,aapl,2018-01-01,0.119,0.000,0.881,neutral
...,...,...,...,...,...,...,...
39301,1003mks,aapl,2022-12-31,0.180,0.000,0.820,neutral
39302,1004hze,aapl,2022-12-31,0.118,0.105,0.776,neutral
39303,1005u7z,aapl,2022-12-31,0.115,0.164,0.721,neutral
39304,1006xh3,aapl,2022-12-31,0.031,0.000,0.969,neutral


In [20]:
# Calculate daily sentiment ratios
sentiment_ratios = (
    apple_posts_df.groupby(['Date', 'sentiment_category'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)
sentiment_ratios

sentiment_category,Date,negative,neutral,positive
0,2018-01-01,0,9,0
1,2018-01-02,0,15,0
2,2018-01-03,0,11,0
3,2018-01-04,0,8,0
4,2018-01-05,0,13,0
...,...,...,...,...
1805,2022-12-27,0,23,0
1806,2022-12-28,0,44,0
1807,2022-12-29,0,40,0
1808,2022-12-30,0,19,0


In [21]:
sentiment_ratios['total_posts'] = sentiment_ratios[['positive', 'negative', 'neutral']].sum(axis=1)
sentiment_ratios['positive_ratio'] = sentiment_ratios['positive'] / sentiment_ratios['total_posts']
sentiment_ratios['negative_ratio'] = sentiment_ratios['negative'] / sentiment_ratios['total_posts']
sentiment_ratios['neutral_ratio'] = sentiment_ratios['neutral'] / sentiment_ratios['total_posts']
sentiment_ratios

sentiment_category,Date,negative,neutral,positive,total_posts,positive_ratio,negative_ratio,neutral_ratio
0,2018-01-01,0,9,0,9,0.0,0.0,1.0
1,2018-01-02,0,15,0,15,0.0,0.0,1.0
2,2018-01-03,0,11,0,11,0.0,0.0,1.0
3,2018-01-04,0,8,0,8,0.0,0.0,1.0
4,2018-01-05,0,13,0,13,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
1805,2022-12-27,0,23,0,23,0.0,0.0,1.0
1806,2022-12-28,0,44,0,44,0.0,0.0,1.0
1807,2022-12-29,0,40,0,40,0.0,0.0,1.0
1808,2022-12-30,0,19,0,19,0.0,0.0,1.0


In [22]:

# Merge the average sentiments and sentiment ratios back into the original DataFrame
combined_sentiments = pd.merge(average_sentiments, sentiment_ratios, on='Date', how='inner')


combined_sentiments.drop(columns=['positive', 'negative', 'neutral', 'total_posts'], inplace=True)
combined_sentiments

,Date,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio
0,2018-01-01,0.081556,0.023889,0.894556,0.0,0.0,1.0
1,2018-01-02,0.129733,0.023333,0.846867,0.0,0.0,1.0
2,2018-01-03,0.071818,0.007000,0.921273,0.0,0.0,1.0
3,2018-01-04,0.144250,0.049125,0.806625,0.0,0.0,1.0
4,2018-01-05,0.059385,0.043308,0.897154,0.0,0.0,1.0
...,...,...,...,...,...,...,...
1805,2022-12-27,0.115478,0.033000,0.851522,0.0,0.0,1.0
1806,2022-12-28,0.052068,0.036750,0.911364,0.0,0.0,1.0
1807,2022-12-29,0.081450,0.045175,0.873450,0.0,0.0,1.0
1808,2022-12-30,0.084789,0.050158,0.865053,0.0,0.0,1.0


In [23]:



#Merging and creating
merged_apple_df = pd.merge(apple_price_df_filtered,combined_sentiments, on='Date', how='left')


merged_apple_df

,Date,Open,High,Low,Close,Volume,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio
0,2018-01-02,40.030320,40.533757,39.818592,40.524345,102223600,0.129733,0.023333,0.846867,0.0,0.0,1.0
1,2018-01-03,40.587864,41.063072,40.453773,40.517288,118071600,0.071818,0.007000,0.921273,0.0,0.0,1.0
2,2018-01-04,40.590212,40.808997,40.481999,40.705486,89738400,0.144250,0.049125,0.806625,0.0,0.0,1.0
3,2018-01-05,40.801943,41.255976,40.710195,41.168934,94640000,0.059385,0.043308,0.897154,0.0,0.0,1.0
4,2018-01-08,41.016029,41.312444,40.917221,41.016029,82271200,0.105111,0.055111,0.839778,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2022-12-23,129.557573,131.041963,128.290894,130.487793,63814900,0.090567,0.037967,0.871533,0.0,0.0,1.0
1255,2022-12-27,130.012822,130.042509,127.380499,128.676865,69007800,0.115478,0.033000,0.851522,0.0,0.0,1.0
1256,2022-12-28,128.320592,129.666440,124.560142,124.728371,85438400,0.052068,0.036750,0.911364,0.0,0.0,1.0
1257,2022-12-29,126.658086,129.122172,126.400797,128.261230,75703700,0.081450,0.045175,0.873450,0.0,0.0,1.0


In [29]:
merged_apple_df[merged_apple_df.isnull().any(axis=1)]

,Date,Open,High,Low,Close,Volume,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio
103,2018-05-31,44.393808,44.633299,44.137717,44.310814,109931200,NaN,NaN,NaN,NaN,NaN,NaN
428,2019-09-16,52.613394,53.193345,52.572315,53.137764,84632400,NaN,NaN,NaN,NaN,NaN,NaN
431,2019-09-19,53.647638,54.070517,53.251340,53.393913,88242400,NaN,NaN,NaN,NaN,NaN,NaN
473,2019-11-18,64.422127,64.817192,64.041610,64.737213,86703200,NaN,NaN,NaN,NaN,NaN,NaN
